<a href="https://colab.research.google.com/github/marinarhianna/python-tutorials/blob/main/ORBYTS_python_additional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Experiment 🧪


*   This notebook lets you explore how small changes to a CNN affects its learning.
*   We will start with a working CNN, and then make one change at a time.

Run each cell, and confirm that our CNN works properly.


In [1]:
# import our modules
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

### 1. Data Preparation

Below we will make up some data to train and test our network on. Our dataset will consist of 3 classes:
* Sine wave (Class 0)
* Square wave (Class 1)
* Random noise (Class 2)

If you're interested, read through the code and have a think how it might work to generate data. If you're not, just run it!

In [2]:
num_samples_per_class = 20
length = 5000

# Class 0: Sine wave
sine_curves = [np.sin(np.linspace(0, np.pi * np.random.uniform(4, 8), length)) + np.random.normal(0, 0.1, length)
               for _ in range(num_samples_per_class)]

# Class 1: Square wave
square_curves = [np.sign(np.sin(np.linspace(0, np.pi * np.random.uniform(4, 8), length))) + np.random.normal(0, 0.1, length)
                 for _ in range(num_samples_per_class)]

# Class 2: Random noise
noise_curves = [np.random.normal(0, 1, length) for _ in range(num_samples_per_class)]


The cell below does all of the data preparation for it to be suitable for training and testing a CNN. Again, don't worry if it doesn't make sense to you, but if you're curious what it is doing, have a read or ask Marina how it works.

In [3]:
# Stack and label
X = np.stack(sine_curves + square_curves + noise_curves)
y = np.array([0]*num_samples_per_class + [1]*num_samples_per_class + [2]*num_samples_per_class)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Convert to tensors and reshape for CNN (N, 1, 5000)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataset and DataLoader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=16)

Now let's have a look at what our data looks like.

In [ ]:
# 📊 Plot one example from each class
for class_label in range(3):
    idx = np.where(y_train == class_label)[0][0]
    plt.figure(figsize=(10, 2))
    plt.plot(X_train[idx])
    plt.title(f"Example from Class {class_label}")
    plt.xlabel("Time step")
    plt.ylabel("Value")
    plt.show()

### 2. Training and Testing our CNN

In [5]:
# 🧠 Base CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(16 * 2500, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

model = SimpleCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 🚂 Training Loop
def train_model():
    train_losses = []
    test_accuracies = []
    for epoch in range(5):
        model.train()
        for xb, yb in train_dl:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
        train_losses.append(loss.item())

        # 🧪 Evaluate
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for xb, yb in test_dl:
                preds = model(xb)
                predicted = torch.argmax(preds, dim=1)
                correct += (predicted == yb).sum().item()
                total += yb.size(0)
        test_accuracies.append(correct / total)
        print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Test Accuracy = {correct/total:.2f}")

    plt.plot(test_accuracies)
    plt.title("Test Accuracy over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.show()

train_model()

# CHALLENGE
Implement each of the changes below 1 at a time, and re-run the cells starting from Section 2 (the cell starting with `# 🧠 Base CNN Model`).

For each change, tell Marina what you observe and why you think it happens.

If it makes the accuracy **worse**, change it back before you go on to the next checklist. If it makes it **better**, keep the change!

### 📜 EXPERIMENT CHECKLIST
1. Increase the size of the dataset. Right now it is 20, so try 50 and 100 and see what happens.
2. Change `Conv1d` kernel size to 9. What happens to the accuracy?
3. Remove the `MaxPool1d` layer. Does it still learn?
4. Change the learning rate to `0.01` or `0.0001`.
5. Right now our dataset is split so training is 80% and testing is 20%. Change the split so training data is 20% and testing is 80%. What happens?
6. Increase number of `epochs` from 5 to 20. Does the model improve more?
7. Try `batch size = 4` vs. `batch size = 64`. How does training speed and accuracy change?
8. Change activation function to `nn.Tanh` instead of `nn.ReLU` – what happens?
9. **Hard:** Add a second convolutional layer to the CNN, with an input of 16 and an output of 32. Does it improve accuracy or overfit?
10. **Extra hard:** Add a dropout layer `(nn.Dropout(0.3))` after ReLU. Does accuracy change?



In [ ]:
# @title Example code for 9.

# Inside SimpleCNN(), add this line after self.conv1
self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)

# Inside the forward pass, add these lines after x = self.conv1(x)
x = self.conv2(x)
x = torch.relu(x)

In [ ]:
# @title Example code for 10.

# Inside SimpleCNN(), add this line
self.dropout = nn.Dropout(0.3)

# Inside the forward pass, add this line just before x = self.pool(x)
x = self.dropout(x)